In [10]:
from bs4 import BeautifulSoup
import requests
import os
from abc import ABC
from PyPDF2 import PdfFileReader, PdfFileWriter, PdfReader
from whoosh import index
from whoosh.index import create_in
from whoosh.fields import Schema, TEXT, ID
import sys
from whoosh.qparser import QueryParser
from whoosh import scoring
from whoosh.index import open_dir
from whoosh.analysis import StandardAnalyzer
import csv
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import wget
import PyPDF2
import glob

start_url = 'https://www.accessdata.fda.gov/scripts/cdrh/cfdocs/cfPMA/pma.cfm?start_search=1&applicant=&tradename=&productcode=&pmanumber=&supplementnumber=&advisorycommittee=&docketnumber=&supplementtype=None&expeditedreview=&center=&ivdproducts=off&combinationproducts=off&decisiondatefrom=01%2F01%2F16&decisiondateto=02%2F02%2F2023&noticedatefrom=&noticedateto=&znumber=&pagenum=500'

def getSummary(URL):
    page_response = requests.get(URL, timeout = 30)
    page_content = BeautifulSoup(page_response.content, "html.parser")
    
    textContent = page_content.find_all(style="text-decoration:underline;")

        #checks if original pma contains a summary
    for element in textContent:
            if(element.text == 'Summary of Safety and Effectiveness'):
                
                
                if int(URL[69:71]) <= 1:
                    return 'http://www.accessdata.fda.gov/cdrh_docs/pdf' + '/' + URL[68:75] + 'B.pdf'
                
                if int(URL[69:71]) >= 70:
                    return 'http://www.accessdata.fda.gov/cdrh_docs/pdf' + '/' + URL[68:75] + 'B.pdf'
                
                #if the pma version is less than 10 the url is formatted a little different
                elif int(URL[69:71]) < 10:
                    return 'http://www.accessdata.fda.gov/cdrh_docs/pdf' + URL[70:71] + '/' + URL[68:75] + 'B.pdf'
                
                return 'http://www.accessdata.fda.gov/cdrh_docs/pdf' + URL[69:71] + '/' + URL[68:75] + 'B.pdf'

    return URL[68:75] + ' does not contain a Summary of Safety and Effectiveness'


def getInfo(ID):
    url = 'https://www.accessdata.fda.gov/scripts/cdrh/cfdocs/cfpma/pma.cfm?id=' + str(ID)
    page_response = requests.get(url, timeout=30)
    page_content = BeautifulSoup(page_response.content, "html.parser")

    advisory_committee = 'none'
    
    textContent = page_content.find_all('tr')
    
    for i in range(9, len(textContent)):
        if 'Advisory Committee' in str(textContent[i]):
            advisory_committee = str(textContent[i]).split('<td align="Left">')[1].split('</td>')[0]
   
    return advisory_committee

#print(getInfo('P190017'))

start_url = 'https://www.accessdata.fda.gov/scripts/cder/daf'
page_response = requests.get(start_url, timeout = 30)
page_content = BeautifulSoup(page_response.content, "html.parser")
textContent = page_content.find_all(style="text-decoration:underline;")

startUrl = 'https://www.accessdata.fda.gov'
i = 0
#looks through all pmas in url
for element in textContent:
    string = str(element)
    deviceURL = startUrl + string[9: 9 + string[9:len(string)].index('\"')] #this pulls up the device specific page 
    getSummary(deviceURL)
    file_url = getSummary(deviceURL)
    
    #if the pma does not have a summary 
    if(len(file_url) == 62):
        print(file_url)
    else:
        r = requests.get(file_url, stream = True) 
        start = len(file_url) - 12
        #wget.download(file_url, 'c:\\Users\\Ivana\\OneDrive\\Desktop\\FDA PMA') #Downloads PDFs summaries directly to local folder
        


# Set the folder path in which we downloaded summaries and the keyword we would like to search for
folder_path = 'c:\\Users\\Ivana\\OneDrive\\Desktop\\FDA PMA'
keywords = ['thermal', 'acoustic', 'electromagnetic', 'inite element', 'cfd','finite element analysis','computational fluid dynamics',
            'fea','computer fluid dynamics','finite element model','computed fluid dynamics','finite element modeling',
            'computational fluid dynamic', 'finite element method','computer fluid dynamic','finite element models',
            'compute fluid dynamic', 'finted element modeled','computing fluid dynamic','finite element analyzed',
            'computation fluid dynamic','numerical analysis','numeric analysis','numerically analyzed','simulation','computer',
            'simulate','computation','simulating','computational','simulations','computing','simulates','compute','simulated',
            'computed','quantitative analysis','mathematical model','quantitative model','mathematical modeling','quantitative models',
            'mathematical models','quantitative modeling','mathematical modeled','quantitatively analyzed','math model','quantitatively modeled',
            'math modeling','physics modeling','mathematics model','physics modeled','mathematics models','math modeled','mathematics modeling',
            'math models','mathematics modeled','mri model','optics model','thermal model','electromagnetic model','optics models',
            'thermal models','electromagnetic models','optics modeled','thermal modeling','electromagnetic modeling',
            'optic model','thermal modeled','electromagnetic modeled','optic models','virtual Population','computed electromagnetic model',
            'optic modeling','virtualization','MRI modeled','modeled','virtualized','magnetic resonance imaging model','computed optics',
            'virtualizations','magnetic resonance imaging models','computed optic','virtual platform','magnetic resonance imaging modeling',
            'computed acoustics','vf','magnetic resonance imaging modeled','computed acoustic','virtual patients','thermal modeling',
            'computing acoustics','virtual reality','thermal model','computing acoustic','vr','thermal modeled','acoustic model',
            'software as medical device','thermal models'
            
]

results = []
keyword_df = pd.DataFrame(columns=['filename','page_number','keyword','text'])

# Loops through all the files in the PMA folder
for filename in os.listdir(folder_path):
    # Checks if the file is a PDF
    if filename.endswith('.pdf'):
        # Open the PDF file 
        with open(os.path.join(folder_path, filename), 'rb') as f:
         
            reader = PdfReader(f)
            
            # Loop through each page of the PDF
            for page in range(len(reader.pages)):
                # Extract the text from the page
                text = reader.pages[page].extract_text()
                
                # Check if the keyword is in the text and print on what page it is
                for keyword in keywords:
                    if keyword in text:
                        # Add the result to the list
                        result = {
                            'filename': filename,
                            'page_number': page+1,
                            'keyword': keyword,
                            'text': text
                        }
                        keyword_df = keyword_df.append(result, ignore_index=True)

   
#print(keyword_df)


pdf_dir =  'c:\\Users\\Ivana\\OneDrive\\Desktop\\FDA PMA'

pdf_files = glob.glob(pdf_dir + '/*.pdf') #Gets all of  the P numbers

#pdf_file_names = [os.path.basename(pdf_file) for pdf_file in pdf_files]

pdf_file_names = [os.path.splitext(os.path.basename(pdf_file))[0][:-1]for pdf_file in pdf_files] #Removes .pdf and the last character "B" in order for the getInfo function to be able to read the Pnumber

#pdf_file_names = [name[:-1]for name in pdf_file_names]



def getInfo(ID):
    url = 'https://www.accessdata.fda.gov/scripts/cdrh/cfdocs/cfpma/pma.cfm?id=' + str(ID)
    page_response = requests.get(url, timeout=30)
    page_content = BeautifulSoup(page_response.content, "html.parser")

    advisory_committee = 'none'
    
    textContent = page_content.find_all('tr')
    
    for i in range(9, len(textContent)):
        if 'Advisory Committee' in str(textContent[i]):
            advisory_committee = str(textContent[i]).split('<td align="Left">')[1].split('</td>')[0]
   
    return advisory_committee

# Create a dataframe for advisory committees
advisory_list = []
pma_nums = pdf_file_names
for pma_num in pma_nums:
    advisory_committee = getInfo(pma_num)
    advisory_list.append({'filename': pma_num+'B.pdf', 'Advisory Committee': advisory_committee})
Advisory_df = pd.DataFrame(advisory_list)

# Merge the two dataframes on the filename column
merged_df = pd.merge(keyword_df, Advisory_df, on='filename')

print(merged_df)

C:\Users\Ivana\AppData\Local\Temp\ipykernel_37188\923134354.py:140: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keyword_df = keyword_df.append(result, ignore_index=True)
C:\Users\Ivana\AppData\Local\Temp\ipykernel_37188\923134354.py:140: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keyword_df = keyword_df.append(result, ignore_index=True)
C:\Users\Ivana\AppData\Local\Temp\ipykernel_37188\923134354.py:140: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keyword_df = keyword_df.append(result, ignore_index=True)
C:\Users\Ivana\AppData\Local\Temp\ipykernel_37188\923134354.py:140: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keyword_df = k

C:\Users\Ivana\AppData\Local\Temp\ipykernel_37188\923134354.py:140: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keyword_df = keyword_df.append(result, ignore_index=True)
C:\Users\Ivana\AppData\Local\Temp\ipykernel_37188\923134354.py:140: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keyword_df = keyword_df.append(result, ignore_index=True)
C:\Users\Ivana\AppData\Local\Temp\ipykernel_37188\923134354.py:140: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keyword_df = keyword_df.append(result, ignore_index=True)
C:\Users\Ivana\AppData\Local\Temp\ipykernel_37188\923134354.py:140: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keyword_df = k

C:\Users\Ivana\AppData\Local\Temp\ipykernel_37188\923134354.py:140: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keyword_df = keyword_df.append(result, ignore_index=True)
C:\Users\Ivana\AppData\Local\Temp\ipykernel_37188\923134354.py:140: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keyword_df = keyword_df.append(result, ignore_index=True)
C:\Users\Ivana\AppData\Local\Temp\ipykernel_37188\923134354.py:140: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keyword_df = keyword_df.append(result, ignore_index=True)
C:\Users\Ivana\AppData\Local\Temp\ipykernel_37188\923134354.py:140: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keyword_df = k

C:\Users\Ivana\AppData\Local\Temp\ipykernel_37188\923134354.py:140: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keyword_df = keyword_df.append(result, ignore_index=True)
C:\Users\Ivana\AppData\Local\Temp\ipykernel_37188\923134354.py:140: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keyword_df = keyword_df.append(result, ignore_index=True)
C:\Users\Ivana\AppData\Local\Temp\ipykernel_37188\923134354.py:140: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keyword_df = keyword_df.append(result, ignore_index=True)
C:\Users\Ivana\AppData\Local\Temp\ipykernel_37188\923134354.py:140: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keyword_df = k

C:\Users\Ivana\AppData\Local\Temp\ipykernel_37188\923134354.py:140: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keyword_df = keyword_df.append(result, ignore_index=True)
C:\Users\Ivana\AppData\Local\Temp\ipykernel_37188\923134354.py:140: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keyword_df = keyword_df.append(result, ignore_index=True)
C:\Users\Ivana\AppData\Local\Temp\ipykernel_37188\923134354.py:140: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keyword_df = keyword_df.append(result, ignore_index=True)
C:\Users\Ivana\AppData\Local\Temp\ipykernel_37188\923134354.py:140: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keyword_df = k

...

C:\Users\Ivana\AppData\Local\Temp\ipykernel_37188\923134354.py:140: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keyword_df = keyword_df.append(result, ignore_index=True)
C:\Users\Ivana\AppData\Local\Temp\ipykernel_37188\923134354.py:140: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keyword_df = keyword_df.append(result, ignore_index=True)
C:\Users\Ivana\AppData\Local\Temp\ipykernel_37188\923134354.py:140: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keyword_df = keyword_df.append(result, ignore_index=True)
C:\Users\Ivana\AppData\Local\Temp\ipykernel_37188\923134354.py:140: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keyword_df = k

C:\Users\Ivana\AppData\Local\Temp\ipykernel_37188\923134354.py:140: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keyword_df = keyword_df.append(result, ignore_index=True)
C:\Users\Ivana\AppData\Local\Temp\ipykernel_37188\923134354.py:140: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keyword_df = keyword_df.append(result, ignore_index=True)
C:\Users\Ivana\AppData\Local\Temp\ipykernel_37188\923134354.py:140: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keyword_df = keyword_df.append(result, ignore_index=True)
C:\Users\Ivana\AppData\Local\Temp\ipykernel_37188\923134354.py:140: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keyword_df = k

C:\Users\Ivana\AppData\Local\Temp\ipykernel_37188\923134354.py:140: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keyword_df = keyword_df.append(result, ignore_index=True)
C:\Users\Ivana\AppData\Local\Temp\ipykernel_37188\923134354.py:140: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keyword_df = keyword_df.append(result, ignore_index=True)
C:\Users\Ivana\AppData\Local\Temp\ipykernel_37188\923134354.py:140: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keyword_df = keyword_df.append(result, ignore_index=True)
C:\Users\Ivana\AppData\Local\Temp\ipykernel_37188\923134354.py:140: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keyword_df = k

C:\Users\Ivana\AppData\Local\Temp\ipykernel_37188\923134354.py:140: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keyword_df = keyword_df.append(result, ignore_index=True)
C:\Users\Ivana\AppData\Local\Temp\ipykernel_37188\923134354.py:140: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keyword_df = keyword_df.append(result, ignore_index=True)
C:\Users\Ivana\AppData\Local\Temp\ipykernel_37188\923134354.py:140: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keyword_df = keyword_df.append(result, ignore_index=True)
C:\Users\Ivana\AppData\Local\Temp\ipykernel_37188\923134354.py:140: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  keyword_df = k

          filename page_number     keyword  \
0     P080028B.pdf           3    acoustic   
1     P120018B.pdf           4  simulation   
2     P120021B.pdf           7    simulate   
3     P120021B.pdf           7   simulated   
4     P130001B.pdf          16  simulation   
...            ...         ...         ...   
1493  P220003B.pdf          11   simulated   
1494  P220006B.pdf           3    computer   
1495  P220006B.pdf           3     compute   
1496  P220006B.pdf          18    simulate   
1497  P220006B.pdf          18   simulated   

                                                   text  \
0     PMA P080028:  FDA Summary of Safety and Effect...   
1     PMA P 120018 :  FDA Summary of Safety and Effe...   
2     PMA P120021:   FDA Summary of Safety and Effec...   
3     PMA P120021:   FDA Summary of Safety and Effec...   
4     PMA P130001:  FDA Summary of Safety and Effect...   
...                                                 ...   
1493    \n \n \n  \n \n \n  \n \n 

In [11]:
display(merged_df)

filename page_number     keyword  \
0     P080028B.pdf           3    acoustic   
1     P120018B.pdf           4  simulation   
2     P120021B.pdf           7    simulate   
3     P120021B.pdf           7   simulated   
4     P130001B.pdf          16  simulation   
...            ...         ...         ...   
1493  P220003B.pdf          11   simulated   
1494  P220006B.pdf           3    computer   
1495  P220006B.pdf           3     compute   
1496  P220006B.pdf          18    simulate   
1497  P220006B.pdf          18   simulated   

                                                   text  \
0     PMA P080028:  FDA Summary of Safety and Effect...   
1     PMA P 120018 :  FDA Summary of Safety and Effe...   
2     PMA P120021:   FDA Summary of Safety and Effec...   
3     PMA P120021:   FDA Summary of Safety and Effec...   
4     PMA P130001:  FDA Summary of Safety and Effect...   
...                                                 ...   
1493    \n \n \n  \n \n \n  \n \n \n \n \n \n \n  \n...   
1494      \n \n \n   \n \n \n \n \n    \n \n \n \n \...   
1495      \n \n \n   \n \n \n \n \n    \n \n \n \n \...   
1496     \n \n  \n \n  \n \n \n \n  \n  \n \n \n \n ...   
1497     \n \n  \n \n  \n \n \n \n  \n  \n \n \n \n ...   

                 Advisory Committee  
0     General &amp; Plastic Surgery  
1                  General Hospital  
2                    Cardiovascular  
3                    Cardiovascular  
4                         Pathology  
...                             ...  
1493                 Cardiovascular  
1494                                 
1495                                 
1496                                 
1497                                 

[1498 rows x 5 columns]

In [12]:
merged_df.to_excel('PMAdata.xlsx', index=False) #converts dataframe to excel spreadsheet